In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mlb-pitch-data-20152018/pitches.csv
/kaggle/input/mlb-pitch-data-20152018/atbats.csv
/kaggle/input/mlb-pitch-data-20152018/games.csv
/kaggle/input/mlb-pitch-data-20152018/2019_games.csv
/kaggle/input/mlb-pitch-data-20152018/2019_atbats.csv
/kaggle/input/mlb-pitch-data-20152018/2019_pitches.csv
/kaggle/input/mlb-pitch-data-20152018/player_names.csv
/kaggle/input/mlb-pitch-data-20152018/ejections.csv


In [2]:
import seaborn as sns

In [3]:
data_raw = pd.read_csv('/kaggle/input/mlb-pitch-data-20152018/pitches.csv')
# Print out size, shape, and column names
print("size: " + str(data_raw.size))
print("shape: " + str(data_raw.shape))
print("columns: " + str(data_raw.columns))

size: 114686160
shape: (2867154, 40)
columns: Index(['px', 'pz', 'start_speed', 'end_speed', 'spin_rate', 'spin_dir',
       'break_angle', 'break_length', 'break_y', 'ax', 'ay', 'az', 'sz_bot',
       'sz_top', 'type_confidence', 'vx0', 'vy0', 'vz0', 'x', 'x0', 'y', 'y0',
       'z0', 'pfx_x', 'pfx_z', 'nasty', 'zone', 'code', 'type', 'pitch_type',
       'event_num', 'b_score', 'ab_id', 'b_count', 's_count', 'outs',
       'pitch_num', 'on_1b', 'on_2b', 'on_3b'],
      dtype='object')


In [4]:
df = data_raw
# dropping useless columns and rows with null values
df = df.drop(["zone", "type_confidence", "pitch_type", "ab_id", "event_num"], axis = 1)
df = df.dropna()
# drop everything except swinging strikes and foul balls
df = df[df.code.isin(['S', 'F'])]
# only 2-strike counts
df = df[df.s_count == 2]
df = df[df.b_count == 3]
print(df.shape)

(55282, 35)


In [6]:
print(df.shape)
df = df.drop(["y0", "type", "b_score", "outs", "pitch_num", "b_count", "s_count", "on_1b", "on_2b", "on_3b"], axis = 1)

(55282, 25)


KeyError: "['y0' 'type' 'b_score' 'outs' 'pitch_num' 'b_count' 's_count' 'on_1b'\n 'on_2b' 'on_3b'] not found in axis"

In [7]:
print(df.columns)

Index(['px', 'pz', 'start_speed', 'end_speed', 'spin_rate', 'spin_dir',
       'break_angle', 'break_length', 'break_y', 'ax', 'ay', 'az', 'sz_bot',
       'sz_top', 'vx0', 'vy0', 'vz0', 'x', 'x0', 'y', 'z0', 'pfx_x', 'pfx_z',
       'nasty', 'code'],
      dtype='object')


In [8]:
# balance out the cases
strikes = df[df.code == 'S']
fouls = df[df.code == 'F']
fouls = fouls[0:len(strikes)]
tojoin = [strikes, fouls]
df = pd.concat(tojoin)
print(strikes.shape)
df.shape

(15199, 25)


(30398, 25)

In [9]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
import sklearn.model_selection as tts

In [10]:
print(df.columns)

Index(['px', 'pz', 'start_speed', 'end_speed', 'spin_rate', 'spin_dir',
       'break_angle', 'break_length', 'break_y', 'ax', 'ay', 'az', 'sz_bot',
       'sz_top', 'vx0', 'vy0', 'vz0', 'x', 'x0', 'y', 'z0', 'pfx_x', 'pfx_z',
       'nasty', 'code'],
      dtype='object')


In [15]:
from sklearn import preprocessing
#creating labelEncoder
le = preprocessing.LabelEncoder()
px_encoded = le.fit_transform(df.px)
pz_encoded= le.fit_transform(df.pz)
start_encoded=le.fit_transform(df.start_speed)
end_encoded=le.fit_transform(df.end_speed)
spin_encoded=le.fit_transform(df.spin_rate)
spin_dir_encoded=le.fit_transform(df.spin_dir)
break_encoded=le.fit_transform(df.break_angle)
break_length_encoded=le.fit_transform(df.break_length)
break_y_encoded=le.fit_transform(df.break_y)
ax_encoded=le.fit_transform(df.ax)
ay_encoded=le.fit_transform(df.ay)
az_encoded=le.fit_transform(df.az)
sz_bot_encoded=le.fit_transform(df.sz_bot)
sz_top_encoded=le.fit_transform(df.sz_top)
vx0_encoded=le.fit_transform(df.vx0)
vy0_encoded=le.fit_transform(df.vy0)
vz0_encoded=le.fit_transform(df.vz0)
x_encoded=le.fit_transform(df.x)
x0_encoded=le.fit_transform(df.x0)
y_encoded=le.fit_transform(df.y)
z0_encoded=le.fit_transform(df.z0)
pfx_x_encoded=le.fit_transform(df.pfx_x)
pfx_z_encoded=le.fit_transform(df.pfx_z)
nasty_encoded = le.fit_transform(df.nasty)

features = np.array(list(zip(px_encoded,pz_encoded,start_encoded,end_encoded, spin_encoded,spin_dir_encoded,break_encoded,
              break_length_encoded,break_y_encoded,ax_encoded,ay_encoded,az_encoded,sz_bot_encoded,
              sz_top_encoded,vx0_encoded,vz0_encoded,x_encoded,x0_encoded,y_encoded, z0_encoded,
              pfx_x_encoded,pfx_z_encoded, nasty_encoded)))
code_encoded = le.fit_transform(df.code) #S = 1, F = 0
print(code_encoded)
print(len(features[0]))

[1 1 1 ... 0 0 0]
23


In [12]:
#df.transpose()[:23].transpose()
features_train, features_test, label_train,label_test = tts.train_test_split(features, code_encoded, test_size = 0.1, random_state = 69)
features_train_2, features_test_2, label_train_2,label_test_2 = tts.train_test_split(df.transpose()[:25].transpose(), code_encoded, test_size = 0.1, random_state = 69)

In [13]:
features_train = np.array(features_train)
labels_train = np.array(label_train)
features_test = np.array(features_test)
labels_test = np.array(label_test)

features_train_2 = np.array(features_train_2)
labels_train_2 = np.array(label_train_2)
features_test_2 = np.array(features_test_2)
labels_test_2 = np.array(label_test_2)

In [14]:
print(features_train.shape)
print(features_train_2.shape)

(27358, 23)
(27358, 25)


In [ ]:
from sklearn.naive_bayes import GaussianNB

#Create a Gaussian Classifier
model = GaussianNB()

# Train the model using the training sets
model.fit(features_train, labels_train)

In [ ]:
predictions = model.predict(features_test)


In [ ]:
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?

print("Accuracy:",metrics.accuracy_score(labels_test, predictions))

# right = 0
# for i in range(len(predictions)):
#     if (predictions[i] == labels_train[i]):
#         right += 1
        
# print(right / len(predictions))

In [29]:
print(labels_train.shape)
print(labels_train_2.shape[0])
print(features_train_2.shape[1])

(27358,)
27358
25


In [ ]:
from sklearn.metrics import confusion_matrix as cm

cm(labels_test,predictions)

# 1009 true negatives
# 514 false positives
# 641 false negatives
# 876 true positives

In [16]:
import keras
from keras import metrics
from keras import regularizers
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.optimizers import Adam, RMSprop
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.utils import plot_model
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import datasets, layers, models

In [57]:
t_model = models.Sequential()
t_model.add(layers.Dense(10000, activation="relu", input_shape=(features_train.shape)))
t_model.add(layers.Dropout(0.40))
t_model.add(layers.Dense(5000, activation="relu"))
t_model.add(layers.Dropout(0.25))
t_model.add(layers.Dense(2500, activation="relu"))
t_model.add(layers.Dropout(0.1))
t_model.add(layers.Dense(1250, activation="relu"))
t_model.add(layers.Dropout(0.1))
t_model.add(layers.Dense(625, activation="relu"))
t_model.add(layers.Dropout(0.1))
t_model.add(layers.Dense(313, activation="relu"))
t_model.add(layers.Dropout(0.1))
t_model.add(layers.Dense(labels_train.shape[0]))
t_model.compile(
    loss="binary_crossentropy",
    optimizer=Adam(lr=0.001),
    metrics=['accuracy'])

In [58]:
history=t_model.fit(features_train, labels_train,epochs=1000,batch_size=128,validation_data=(features_test,labels_test))
test_loss, test_acc = t_model.evaluate(features_test,labels_test)

Epoch 1/1000
214/214 [==============================] - 6s 25ms/step - loss: 7.6646 - accuracy: 0.0000e+00 - val_loss: 7.6673 - val_accuracy: 0.0000e+00
Epoch 2/1000
214/214 [==============================] - 5s 24ms/step - loss: 7.6690 - accuracy: 0.0000e+00 - val_loss: 7.6673 - val_accuracy: 0.0000e+00
Epoch 3/1000
214/214 [==============================] - 5s 24ms/step - loss: 7.6681 - accuracy: 0.0000e+00 - val_loss: 7.6674 - val_accuracy: 0.0000e+00
Epoch 4/1000
214/214 [==============================] - 5s 24ms/step - loss: 7.6672 - accuracy: 0.0000e+00 - val_loss: 7.6676 - val_accuracy: 0.0000e+00
Epoch 5/1000
214/214 [==============================] - 5s 24ms/step - loss: 7.6663 - accuracy: 0.0000e+00 - val_loss: 7.6673 - val_accuracy: 0.0000e+00
Epoch 6/1000
214/214 [==============================] - 5s 24ms/step - loss: 7.6673 - accuracy: 0.0000e+00 - val_loss: 7.6675 - val_accuracy: 0.0000e+00
Epoch 7/1000
214/214 [==============================] - 5s 24ms/step - loss: 7.665

KeyboardInterrupt: 

In [ ]:
from sklearn.neural_network import MLPRegressor
fear = MLPRegressor(random_state=1, max_iter=100).fit(features_train, labels_train)
fear.score(features_test, labels_test)

In [ ]:
degree = 5
polyreg = make_pipeline(PolynomialFeatures(degree),LinearRegression())
polyreg.fit(features_train,labels_train)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.scatter(features_train,labels_train)
plt.plot(features_test,polyreg.predict(features_test),color="black")
plt.title("Polynomial regression with degree "+str(degree))
plt.show()

In [ ]:
# convert pitch outcomes to numbers so they can be plotted
# higher number = better outcome
# outcomes = {'S': 1, 'C': 1, 'W': 1, 'T': 0.75, 'F': 0.5, 'X': 0.25, 'D': 0, 'E': 0 }
# df['code_num'] = df['code'].map(outcomes)
# df = df.drop(['code'], axis = 1) 
sns.displot(df.code_num)